# Inference Stress Test

## Multi-threaded inference test

This is a tiny script that attempts slam one of our deployed endpoints with requests. If you are testing, say, inference speed on **vLLM** you absolutely
**must** do something like this to even get a valid result; you can't take advantage of continuous batching without many requests running



In [ ]:
# we expect openai, specifically 0.28.1
!pip install --upgrade "openai>=0.28.1,<1.0"

In [ ]:
# for a gentle test, set this to a positive number. If you do, we will only run one at a time. To run the actual stress test, leave at -1
PROMPTS_TO_RUN = 5
# PROMPTS_TO_RUN = -1

# import to look up deployments
from kamiwaza.serving.services import ServingService
serving_service = ServingService()


In [ ]:
import time
import concurrent.futures
import openai

import logging
logging.basicConfig(level=logging.WARNING)

# be less noisy
# Set the level for all loggers to WARNING
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

In [ ]:
import random

myprompts = [
    "write me code in python to efficiently print prime numbers 0..1b"
    "write me a fastapi server that handles notes and can get/put them",
    "write me code that demonstrates a simple version of doing a 3-param linear regression model for home prices with price, sqft, rooms to predict prices in that market given sqft/rooms, training set in /tmp/trainingdata.json named 'price', 'sqft', 'rooms' as columns, count the number of rows in the code and split 90/10 training/testing",
    "write me a fastapi server that has endpoints for playing tic tac toe",
    "write me a fastapi server that has endpoints for playing chess; ensure all moves are valid; use chess notation for moves",
    "write me a list of the 50 most influential ai/compsci pioneers, 1 per line, numbered 1..50, with 10-12 words on why they belong on the list; you must not repeat any name",
    "write me an example of factoring an exceedingly large prime with a quantum computer",
    "show me three examples of messy python code that can be improved to be more idiomatic/pythonic with a comprehension",
    "come up with a sqlalchemy schema for a D&D 5e character"
]

prompts = [
    "Python script that scrapes weather data from a website and saves it in a CSV file",
    "create a Django app for a blog where users can post, edit, and delete their blogs",
    "write a Python function to convert a given text into Morse Code",
    "design a Python program that uses machine learning to predict stock market prices",
    "develop a command-line tool in Python for renaming files in bulk according to a specified pattern",
    "write a Python script to automate login to a website and download a specific file",
    "create a Flask API for a todo list application with CRUD operations",
    "write a Python script that uses OpenCV for facial recognition",
    "develop a Python-based text editor with basic functionalities like open, edit, and save files",
    "create a Python script for a basic chatbot that can answer FAQs",
    "write a Python program that solves Sudoku puzzles",
    "develop a web scraper in Python to collect data from multiple pages of a website",
    "write a Python script to monitor and log CPU and memory usage over time",
    "create a Python program that encrypts and decrypts text using a custom cipher",
    "write a Python script that visualizes data from a CSV file using MatpZlotlib",
    "develop a Python-based system for tracking and managing inventory",
    "create a Python program to analyze and plot cryptocurrency trends",
    "write a Python script that automates sending emails with attachments",
    "develop a Python application that converts speech to text",
    "write a Python-based calculator for complex mathematical expressions",
    "create a Python script to compare two text files and highlight differences",
    "develop a Python program that uses natural language processing to summarize text",
    "write a Python script for a password generator that creates strong, unique passwords",
    "create a Python-based tool for organizing and renaming photos based on date and location",
    "develop a Python script that tracks prices of products on e-commerce websites and sends alerts",
    "write a Python program for a simple expense tracker",
    "create a Python script to automate data entry into a web form",
    "develop a Python tool for visualizing geographic data on a map",
    "write a Python script to analyze and predict weather patterns",
    "create a Python-based server monitoring tool that sends alerts for downtime",
    "write a Python program that automates file backup to cloud storage",
    "develop a Python script for real-time currency conversion",
    "write a Python-based tool for sentiment analysis of social media posts",
    "create a Python script to manage and rotate log files",
    "develop a Python program for an automated reminder system",
    "write a Python script to convert PDF files to text",
    "create a Python-based tool for detecting plagiarism in text",
    "develop a Python script for automated image resizing and optimization",
    "write a Python program for a simple version control system",
    "create a Python script that simulates a rolling dice game",
    "develop a Python-based email parser that extracts and organizes key information",
    "write a Python script for a system health check that reports on key metrics",
    "create a Python program for a virtual assistant that can perform basic tasks",
    "develop a Python script that merges and splits PDF files",
    "write a Python program for a digital clock with additional features like timer and stopwatch",
    "create a Python script to manage tasks and deadlines for a project",
    "develop a Python-based tool for audio file conversion and editing",
    "write a Python script for a text-based adventure game",
    "create a Python program for a basic inventory management system for a small shop",
    "develop a Python script for tracking and analyzing personal fitness data"
]

additional_prompts = [
    "write a Python script to organize and analyze personal finance data from bank statements",
    "create a Python-based GUI application for a digital address book",
    "develop a Python script that automates the creation of PowerPoint presentations from a text outline",
    "write a Python program to generate music playlists based on mood analysis",
    "create a Python script for a basic file encryption and decryption tool",
    "develop a Python-based web application for real-time sports scores and statistics",
    "write a Python program to automate the process of image tagging using AI",
    "create a Python script for real-time chat translation for multiple languages",
    "develop a Python tool for tracking and managing personal health data",
    "write a Python-based system for managing library books and members",
    "create a Python script to convert handwriting to text using machine learning",
    "develop a Python program for a restaurant reservation system",
    "write a Python script for a tool that optimizes travel routes and schedules",
    "create a Python-based application for tracking and managing event tickets",
    "develop a Python script for a workout planner and tracker",
    "write a Python program for a real-time collaborative whiteboard",
    "create a Python script to generate and analyze fantasy sports teams",
    "develop a Python-based tool for home automation control",
    "write a Python script for an automated plant watering system",
    "create a Python program for a virtual reality tour guide",
    "develop a Python script for a mobile app backend with REST API",
    "write a Python script for sentiment analysis in customer reviews",
    "create a Python-based tool for tracking wildlife movements",
    "develop a Python program for a self-learning flashcard application",
    "write a Python script for an augmented reality shopping assistant",
    "create a Python script for a tool that converts blog posts into podcasts",
    "develop a Python-based system for managing and organizing recipes",
    "write a Python program for a 2D game engine",
    "create a Python script for a voice-activated calculator",
    "develop a Python tool for analyzing and optimizing network traffic",
    "write a Python script for a tool that summarizes scientific papers",
    "create a Python program for a task prioritization and scheduling assistant",
    "develop a Python script for a tool that generates diet plans based on nutritional needs",
    "write a Python script for a vehicle maintenance and service tracker",
    "create a Python-based e-commerce website backend",
    "develop a Python script for a tool that detects and reports cybersecurity threats",
    "write a Python program for a virtual pet that evolves based on user interactions",
    "create a Python script for a tool that helps in learning new languages",
    "develop a Python program for an online auction system",
    "write a Python script for a tool that organizes and displays family trees",
    "create a Python-based application for tracking public transport in real-time",
    "develop a Python script for a custom news aggregator",
    "write a Python program for a tool that assists in video editing",
    "create a Python script for an app that matches tutors with students",
    "develop a Python-based system for tracking and predicting market trends",
    "write a Python script for a virtual reality workout game",
    "create a Python program for a tool that helps with meditation and mindfulness",
    "develop a Python script for an AI-powered personal fashion stylist",
    "write a Python script for a tool that automates essay writing",
    "create a Python program for a virtual reality art gallery"
]
prompts = prompts + myprompts + additional_prompts
random.shuffle(prompts)
len(prompts)

In [ ]:

import subprocess
import os
import re

deployments = serving_service.list_deployments(with_names=True)
valid_deployments = []
for deployment in deployments:
    if deployment.status == 'STOPPED':
        continue
    if not deployment.instances:
        continue

    valid_deployments.append(deployment)
    print(f"Found a deployment of {deployment.m_name} - using it")
    break


In [ ]:
openai.api_key = "local"
openai.api_base = f"http://localhost:{deployment.instances[0].listen_port}/v1"
print(f"Endpoint: {openai.api_base}")
openai.verify_ssl_certs = False

In [ ]:

def fetch_response(prompt, chat_params):
    start_time = time.time()
    params = chat_params.copy()  # Assuming chat_params is a predefined dictionary
    params['messages'] = [{'role': 'system', 'content': 'You are an elite AI assistant, expert at coding. You always do everything you can to be helpful to the user'}, {'role': 'user', 'content': prompt}]
    params['timeout'] = 600
    response = openai.ChatCompletion.create(**params)
    end_time = time.time()
    usage = response.get('usage', {'prompt_tokens': 0, 'total_tokens': 0, 'completion_tokens': 0})
    return prompt, response, end_time - start_time, usage

def main(prompts, chat_params):
    start_time = time.time()
    total_prompt_tokens, total_completion_tokens, total_tokens = 0, 0, 0
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(fetch_response, prompt, chat_params): prompt for prompt in prompts}
        for future in concurrent.futures.as_completed(futures):
            prompt, response, duration, usage = future.result()
            total_prompt_tokens += usage['prompt_tokens']
            total_completion_tokens += usage['completion_tokens']
            total_tokens += usage['total_tokens']
            print(f"Response to {prompt}:\n\n{response['choices'][0]['message']['content']}\n")
            print(f"Time taken: {duration} seconds\n")
            print("---\n")
    total_time = time.time() - start_time
    print(f"Total execution time: {total_time} seconds")
    num_prompts = len(prompts)
    avg_prompt_tokens = total_prompt_tokens / num_prompts
    avg_completion_tokens = total_completion_tokens / num_prompts
    avg_total_tokens = total_tokens / num_prompts
    print(f"Total execution time: {total_time} seconds")
    print(f"Total prompt tokens: {total_prompt_tokens}, Average: {avg_prompt_tokens}")
    print(f"Total completion tokens: {total_completion_tokens}, Average: {avg_completion_tokens}")
    print(f"Total tokens: {total_tokens}, Average Per Prompt: {avg_total_tokens}, Avg Speed: {total_tokens/total_time} Tokens/s")

# Notes

Note that we use our thread executor pool here to dispatch all the prompts at once, with a 10-minute timeout.

That means your system and engine will need to respond to all the prompts in that 10-minute window.

This is obviously very model-dependent; but say, vLLM on an RTX4090 or A100 running DeepSeek-Coder-33B (-AWQ for the 4090) will take ~2 minutes, doing around 500-600tok/s and typically generating ~60k tokens or so total.

Serially executing on a llamacpp deployment on OSX, it will certainly timeout. (The same model on my M2.max will do around ~25t/s iirc, but that means >40 miuntes).

We haven't tested the large batch against llamacpp and it may have issues with completing the entire set; thus the default of 5 in the notebook; our OpenHermes example zips along on an m2.max doing ~60t/s, but parallelizing 100+ queries is too much.


In [ ]:
main(prompts=prompts[0:PROMPTS_TO_RUN if PROMPTS_TO_RUN>0 else None], chat_params={'model': 'model'})